# InfernoSaber 

Hi, here you can generate beat saber songs using the new automapper [InfernoSaber](https://github.com/fred-brenner/BeatSaber-Extraction)

Thanks to guillefix [DeepSaber](https://github.com/oxai/deepsaber) for the notebook layout.


**Run this every time you start the Colab session:**

Run the "Prepare dependencies" cell

**To generate each song:**

Follow the instructions below to upload song and generate levels

It should typically take 20 to 40 seconds depending on length of song (with GPU).

In [ ]:
#@title Connect Google Drive (double click to see code)
#@markdown This will connect colab to your Google Drive (will ask you for permission). Make sure you have more than a 1 GB free on your Google Drive (to be safe)

#@markdown It will then create a folder to save the InfernoSaber depencies
#@markdown or if it's already created it will use this name to find the folder


from google.colab import drive, files
drive.mount('/content/drive')
import os
from os import mkdir, path
def cd(dir):
  os.chdir(dir)
infernosaber_folder_name = 'InfernoSaber'
working_dir=!pwd
working_dir=working_dir[0]
if working_dir=="/content":
  if not path.exists("drive/My Drive/"+infernosaber_folder_name):
    mkdir("drive/My Drive/"+infernosaber_folder_name)
  cd("drive/My Drive/"+infernosaber_folder_name)

#@markdown **run this cell** by pressing the play arrow on the left (or press ctrl+enter or shift+enter while having this cell selected)

Mounted at /content/drive


In [ ]:
#@title Prepare InfernoSaber and dependencies
#@markdown Run this cell to download deepsaber to your drive (only will if it hasn't downloaded already), and prepare dependencies

if len(os.listdir()) > 0:
  print("Code already setup")
else:
  print("Download pretrained package.")
  !wget -O 'InfernoSaber.zip' 'https://docs.google.com/uc?export=download&id=1l2xRBiTzCORcV16wR9H9BChncJpALinV&confirm=t'
  import zipfile
  with zipfile.ZipFile('InfernoSaber.zip', 'r') as zip_ref:
    zip_ref.extractall('')
  os.remove('InfernoSaber.zip')
  print("Finished code setup")

# install needed python modules in notebook
print("Installing music dependencies...")
!pip install scikit-learn==1.0.2 aubio pydub keras-tcn --quiet
# tensorflow related modules are currently installed on every colab notebook
print("Finished environment setup. Ready for takeoff!")

Download pretrained package.
--2023-03-22 19:55:49--  https://docs.google.com/uc?export=download&id=1l2xRBiTzCORcV16wR9H9BChncJpALinV&confirm=t
Resolving docs.google.com (docs.google.com)... 74.125.200.100, 74.125.200.138, 74.125.200.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.200.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-a4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/h57r8u9kf3nf5ntmd48s01oo1jcmdfi8/1679514900000/05895113502157697689/*/1l2xRBiTzCORcV16wR9H9BChncJpALinV?e=download&uuid=fd0586bf-b2d2-4346-95ef-28212173791e [following]
--2023-03-22 19:55:49--  https://doc-0c-a4-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/h57r8u9kf3nf5ntmd48s01oo1jcmdfi8/1679514900000/05895113502157697689/*/1l2xRBiTzCORcV16wR9H9BChncJpALinV?e=download&uuid=fd0586bf-b2d2-4346-95ef-28212173791e
Resolving doc-0c-a4-docs.googleusercontent.com (doc-0c-a4-docs.googleusercon

## Generate

In [ ]:
#@title Local song upload
#@markdown Upload a song (MP3, MP4, OGG, EGG supported) or

#@markdown You can skip this step and directly copy the music into the Drive folder 
#@markdown "/content/drive/MyDrive/InfernoSaber/Data/prediction/songs_predict/"

#@markdown You choose multiple songs at once or rerun this cell to import multiple songs

#@markdown You can choose to delete 1.) old input songs, 2.) old output maps

# Optional: delete all files in directory
song_path = 'Data/prediction/songs_predict/'
del_flag = input("Delete previous input songs? (y or n)")
if del_flag.lower() == 'y':
  for f in os.listdir(song_path):
    if not f.startswith('Put your music files HERE.txt'):
      os.remove(song_path + f)
del_flag = input("Delete previous output maps? (y or n)")
if del_flag.lower() == 'y':
  map_path = 'Data/prediction/new_map/'
  for f in os.listdir(map_path):
    if f not in ["cover.jpg", "Output directory.txt"]:
      if os.path.isfile(map_path + f):
        os.remove(map_path + f)
      else:
        import shutil
        shutil.rmtree(map_path + f)

# save new music file(s)
res=files.upload()
filename=list(res.keys())
for song_file in filename:
  os.rename(song_file, song_path + song_file)

Delete previous input songs? (y or n)y
Delete previous output maps? (y or n)y


Saving Cornetto (Dumme Jungs Remix).egg to Cornetto (Dumme Jungs Remix).egg


In [ ]:
#@title Youtube song download

#@markdown Library by [pytube](https://github.com/rmerzouki/pytube)

#@markdown You can rerun this cell to import multiple songs

!pip install pytube --quiet
from pytube import YouTube

song_path = 'Data/prediction/songs_predict/'

download_from_youtube = True #@param {type:"boolean"}
link = 'https://www.youtube.com/watch?v=fr-xxBY_BdQ&ab_channel=CamelliaOfficial' #@param {type:"string"}

if download_from_youtube:
  if (link.startswith("https://youtu.be/")): # fix
    link = link.replace("https://youtu.be/", "https://www.youtube.com/watch?v=")

  # Create a YouTube object
  video = YouTube(link)
  audio_streams = video.streams.filter(only_audio=True)
  # Select the first audio stream from the list (which is usually the one with the highest audio quality)
  audio_stream = audio_streams[0]
  audio_stream.download(output_path=song_path)


In [ ]:
#@title Specify mapper parameters
#@markdown Set the maximum difficulty and further parameters by adjusting in the code (double click to see code).

# (make sure to put all values in paranthese)

# recommended: 15 for hard, 25 for Expert+; set between 5 for easy difficulty to 40 for expert++
os.environ['MAX_SPEED'] = '30'

# set quick start behaviour: 0: deactivated, around 1.0: low, around 2.0: high (higher values may introduce glitter in the beginning)
os.environ['quick_start'] = '1'


In [ ]:
#@title Generate level for song
#@markdown Output will be put in the new map folder (both zip and extracted): "/content/drive/MyDrive/InfernoSaber/Data/prediction/new_map" 
#@markdown

#@markdown Old maps will stay except for 1) same song again 2) removed in step "Local song upload"

#@markdown Download file by right click and press download or go into the google drive to download all.

# start main script
!python "Code/main.py"

# files.download("scripts/generation/generated/"+generated_filename)

2023-03-22 20:17:00.913437: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 20:17:02.171649: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-22 20:17:02.171834: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-22 20:17:02.171864: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

You can have a view on the mapped songs at "/content/drive/MyDrive/InfernoSaber/Data/prediction/new_map" 

You can upload the generated zip file to view the map online here: https://skystudioapps.com/bs-viewer/
(thanks to +1 Rabbit).